# Initialization

Test notebook for the C-MAPPS benchmark. Get best parameters for each dataset. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import time
import numpy as np
import csv
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tunable_model import SequenceTunableModelRegression

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

from scipy.optimize import differential_evolution

import CMAPSAuxFunctions

Using TensorFlow backend.


# Define the model

In [2]:
#K.clear_session()  #Clear the previous tensorflow graph

l2_lambda_regularization = 0.20
l1_lambda_regularization = 0.10

def RULmodel_SN(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create theTunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):

    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = RULmodel_SN(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        """nFeatures = len(selected_features)
        shapeLSTM = (window_size, nFeatures)
        model = RULmodel_LSTM(shapeLSTM)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)"""
        pass
    else:
        pass

    return model

In [6]:
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = get_compiled_model(shapeSN, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps)
#tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', modelRULLSTM, lib_type='keras', data_handler=dHandler_cmaps)
tModel.data_scaler = min_max_scaler

# Define objective function

Define the function that evaluates each set of data-related params and returns the RMSE as value.

In [7]:
def nn_optmize_fun(x, tModel, verbose=0, epochs=250, saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)

    #load the data using the selected parameters
    tModel.data_handler.sequence_length = x[0]
    #tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
    tModel.data_handler.sequence_stride = x[1]
    tModel.data_handler.max_rul = x[2]
    
    tModel.load_data(unroll=True, verbose=0, cross_validation_ratio=0)
    
    if iterations == 0:
        print("Training model")
    
    #Create new model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    nFeatures = len(selected_features)
    model_shape = nFeatures*tModel.data_handler.sequence_length
    modelRULSN = get_compiled_model(model_shape, model_type='ann')
    tModel.change_model('ModelRUL_SN_1', modelRULSN, 'keras')
    tModel.epochs = 20
    
    #Train model
    tModel.train_model(learningRate_scheduler=lrate, verbose=0)
    time = tModel.train_time
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluate_model(["rmse", "rhs"], round=2)
    cScores = tModel.scores
    rmse = cScores['rmse']
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    print(msgStr)
    row = x.tolist() + [rmse, rhs]
    
    if saveToFile is not None:
        #print(msgStr)
        writer = csv.writer(saveToFile)
        #row = x.append(rmse)
        #row = x.append(rhs)
        writer.writerow(row)
        #saveToFile.write(msgStr)
    else:
        print(row)
    
    #Return RMSE as the performance metric to steer the search
    return rmse
    

# Optimize the parameters for the NN using DE

In [8]:
#Optimize the parameters for the NN using DE

#maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
maxWindowSize = {'1':30, '2':20} #Do it only for datasets 1 and 2
totalTime = {'1':0, '2':0, '3':0, '4':0}
results = {'1':0, '2':0, '3':0, '4':0}

#datasetNumber = '1'

selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']



for datasetNumber in maxWindowSize:
    
    print("Tunning for dataset "+datasetNumber)
    file = open("results/MLP/intermediateResults_refactorized_"+datasetNumber+".csv", "w")

    windowSizeBounds = [1,maxWindowSize[datasetNumber]]
    windowStrideBounds = [1,10]
    constantRULBounds = [90,140]

    bounds = [windowSizeBounds, windowStrideBounds, constantRULBounds]
    #bounds = [windowStrideBounds, constantRULBounds] #Optimize only 2 variabes
    
    tModel.data_handler.change_dataset(datasetNumber)

    startTime = time.clock()
    tempResults = differential_evolution(nn_optmize_fun, bounds, 
                                     args=(tModel, 0, 20, file, 1),
                                    strategy='best1bin', maxiter=30, popsize=4, disp=True, polish=False)
    results[datasetNumber] = tempResults
    endTime = time.clock()

    file.close()
    totalTime[datasetNumber] = endTime - startTime

Tunning for dataset 1
100/100 [==============================] - 0s 240us/step
The model variables are [ 1  1 91]	The scores are: [RMSE:27.1030, RHS:13.3821]

100/100 [==============================] - 0s 769us/step
The model variables are [11  8 96]	The scores are: [RMSE:51.4500, RHS:225.5656]

100/100 [==============================] - 0s 239us/step
The model variables are [ 28   9 121]	The scores are: [RMSE:44.3276, RHS:139.5245]

100/100 [==============================] - 0s 239us/step
The model variables are [ 17   4 128]	The scores are: [RMSE:23.0976, RHS:12.7067]

100/100 [==============================] - 0s 261us/step
The model variables are [ 14   6 117]	The scores are: [RMSE:28.9715, RHS:22.3208]

100/100 [==============================] - 0s 232us/step
The model variables are [  6   8 101]	The scores are: [RMSE:57.3006, RHS:319.2007]

100/100 [==============================] - 0s 238us/step
The model variables are [  4   5 108]	The scores are: [RMSE:33.3380, RHS:36.2867]

1

100/100 [==============================] - 0s 239us/step
The model variables are [ 11   7 111]	The scores are: [RMSE:35.7596, RHS:49.0774]

100/100 [==============================] - 0s 251us/step
The model variables are [28  8 99]	The scores are: [RMSE:50.4466, RHS:236.2431]

differential_evolution step 4: f(x)= 16.2173
100/100 [==============================] - 0s 254us/step
The model variables are [ 21   1 126]	The scores are: [RMSE:17.5841, RHS:5.5456]

100/100 [==============================] - 0s 236us/step
The model variables are [ 19   7 122]	The scores are: [RMSE:36.9377, RHS:63.0353]

100/100 [==============================] - 0s 242us/step
The model variables are [ 19   2 121]	The scores are: [RMSE:19.6069, RHS:11.6211]

100/100 [==============================] - 0s 246us/step
The model variables are [24  5 94]	The scores are: [RMSE:28.4440, RHS:16.4740]

100/100 [==============================] - 0s 246us/step
The model variables are [ 10   2 110]	The scores are: [RMSE:21.0

100/100 [==============================] - 0s 242us/step
The model variables are [ 23   1 127]	The scores are: [RMSE:16.1487, RHS:4.0937]

100/100 [==============================] - 0s 238us/step
The model variables are [ 24   1 125]	The scores are: [RMSE:15.8136, RHS:3.8613]

100/100 [==============================] - 0s 264us/step
The model variables are [ 22   1 126]	The scores are: [RMSE:16.6478, RHS:4.2128]

100/100 [==============================] - 0s 237us/step
The model variables are [ 25   2 133]	The scores are: [RMSE:18.6422, RHS:9.1000]

100/100 [==============================] - 0s 456us/step
The model variables are [ 24   1 120]	The scores are: [RMSE:16.1812, RHS:3.6736]

differential_evolution step 9: f(x)= 15.7528
100/100 [==============================] - 0s 299us/step
The model variables are [ 24   1 125]	The scores are: [RMSE:15.3144, RHS:3.4619]

100/100 [==============================] - 0s 398us/step
The model variables are [ 24   2 125]	The scores are: [RMSE:18.7

100/100 [==============================] - 0s 241us/step
The model variables are [ 24   1 127]	The scores are: [RMSE:15.3268, RHS:3.4265]

100/100 [==============================] - 0s 238us/step
The model variables are [ 24   1 125]	The scores are: [RMSE:16.1075, RHS:4.0771]

100/100 [==============================] - 0s 248us/step
The model variables are [ 25   1 126]	The scores are: [RMSE:16.2552, RHS:4.2217]

100/100 [==============================] - 0s 258us/step
The model variables are [ 24   1 125]	The scores are: [RMSE:15.9540, RHS:3.9521]

100/100 [==============================] - 0s 239us/step
The model variables are [ 24   1 129]	The scores are: [RMSE:15.2417, RHS:3.4513]

100/100 [==============================] - 0s 250us/step
The model variables are [ 23   1 130]	The scores are: [RMSE:16.7812, RHS:4.6392]

100/100 [==============================] - 0s 254us/step
The model variables are [ 24   1 129]	The scores are: [RMSE:15.6221, RHS:3.6461]

100/100 [==================

100/100 [==============================] - 0s 242us/step
The model variables are [ 24   1 129]	The scores are: [RMSE:15.9997, RHS:4.0777]

100/100 [==============================] - 0s 245us/step
The model variables are [ 24   1 125]	The scores are: [RMSE:16.0147, RHS:4.0682]

100/100 [==============================] - 0s 235us/step
The model variables are [ 25   2 132]	The scores are: [RMSE:18.2923, RHS:8.0991]

100/100 [==============================] - 0s 248us/step
The model variables are [ 23   1 128]	The scores are: [RMSE:15.9587, RHS:4.0496]

100/100 [==============================] - 0s 242us/step
The model variables are [ 24   2 129]	The scores are: [RMSE:19.2401, RHS:12.0608]

100/100 [==============================] - 0s 247us/step
The model variables are [ 23   1 126]	The scores are: [RMSE:16.6853, RHS:4.5514]

100/100 [==============================] - 0s 238us/step
The model variables are [ 23   2 132]	The scores are: [RMSE:20.5183, RHS:20.1459]

100/100 [================

100/100 [==============================] - 0s 248us/step
The model variables are [ 24   1 131]	The scores are: [RMSE:16.0913, RHS:4.3499]

differential_evolution step 23: f(x)= 15.2417
100/100 [==============================] - 0s 234us/step
The model variables are [ 24   2 127]	The scores are: [RMSE:18.4543, RHS:8.4562]

100/100 [==============================] - 0s 241us/step
The model variables are [ 24   1 127]	The scores are: [RMSE:16.6562, RHS:4.4236]

100/100 [==============================] - 0s 248us/step
The model variables are [ 24   1 125]	The scores are: [RMSE:16.0714, RHS:3.7236]

100/100 [==============================] - 0s 246us/step
The model variables are [ 24   2 130]	The scores are: [RMSE:18.8772, RHS:9.8133]

100/100 [==============================] - 0s 250us/step
The model variables are [ 24   1 129]	The scores are: [RMSE:15.9226, RHS:3.9484]

100/100 [==============================] - 0s 254us/step
The model variables are [ 23   1 130]	The scores are: [RMSE:16.

100/100 [==============================] - 0s 249us/step
The model variables are [ 24   1 129]	The scores are: [RMSE:15.8770, RHS:4.2733]

100/100 [==============================] - 0s 252us/step
The model variables are [ 24   1 128]	The scores are: [RMSE:16.0147, RHS:3.8245]

100/100 [==============================] - 0s 241us/step
The model variables are [ 24   1 131]	The scores are: [RMSE:16.5523, RHS:4.6445]

100/100 [==============================] - 0s 244us/step
The model variables are [ 24   2 132]	The scores are: [RMSE:21.2601, RHS:23.5068]

differential_evolution step 28: f(x)= 15.2417
100/100 [==============================] - 0s 255us/step
The model variables are [ 24   1 130]	The scores are: [RMSE:16.0786, RHS:4.3443]

100/100 [==============================] - 0s 429us/step
The model variables are [ 23   1 132]	The scores are: [RMSE:16.1096, RHS:4.0247]

100/100 [==============================] - 0s 260us/step
The model variables are [ 24   1 125]	The scores are: [RMSE:16

259/259 [==============================] - 0s 116us/step
The model variables are [ 10   3 118]	The scores are: [RMSE:47.5017, RHS:310.6583]

259/259 [==============================] - 0s 118us/step
The model variables are [13  1 93]	The scores are: [RMSE:39.8178, RHS:299.6480]

259/259 [==============================] - 0s 115us/step
The model variables are [17  1 96]	The scores are: [RMSE:38.6779, RHS:246.1158]

259/259 [==============================] - 0s 111us/step
The model variables are [ 14   1 139]	The scores are: [RMSE:31.7041, RHS:165.9355]

259/259 [==============================] - 0s 110us/step
The model variables are [ 13   6 130]	The scores are: [RMSE:57.1488, RHS:1526.8605]

259/259 [==============================] - 0s 108us/step
The model variables are [  2   1 109]	The scores are: [RMSE:38.1500, RHS:155.9703]

259/259 [==============================] - 0s 109us/step
The model variables are [ 17   7 139]	The scores are: [RMSE:57.6477, RHS:2351.7200]

differential_evol

259/259 [==============================] - 0s 110us/step
The model variables are [  6   9 111]	The scores are: [RMSE:60.3594, RHS:1776.7820]

259/259 [==============================] - 0s 111us/step
The model variables are [ 18   1 127]	The scores are: [RMSE:32.3637, RHS:112.4766]

259/259 [==============================] - 0s 115us/step
The model variables are [ 16   1 120]	The scores are: [RMSE:32.5395, RHS:93.1002]

259/259 [==============================] - 0s 113us/step
The model variables are [12  1 94]	The scores are: [RMSE:39.7880, RHS:318.3967]

259/259 [==============================] - 0s 106us/step
The model variables are [  5   1 120]	The scores are: [RMSE:34.9759, RHS:201.2716]

259/259 [==============================] - 0s 113us/step
The model variables are [ 17   8 139]	The scores are: [RMSE:57.5580, RHS:2123.4765]

259/259 [==============================] - 0s 106us/step
The model variables are [ 16   1 139]	The scores are: [RMSE:31.3339, RHS:103.6070]

259/259 [======

259/259 [==============================] - 0s 111us/step
The model variables are [ 16   1 139]	The scores are: [RMSE:31.6187, RHS:85.7582]

differential_evolution step 11: f(x)= 30.954


In [9]:
print("Total time {}".format(totalTime))
print(results)

Total time {'1': 2090.3476130000004, '2': 1592.4739800000002, '3': 0, '4': 0}
{'1':      fun: 15.241719063150324
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 372
     nit: 30
 success: False
       x: array([  24.16284533,    1.95561666,  129.90776725]), '2':      fun: 30.95400748127097
 message: 'Optimization terminated successfully.'
    nfev: 144
     nit: 11
 success: True
       x: array([  17.19795954,    1.29900828,  139.28000228]), '3': 0, '4': 0}
